In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd

In [2]:
class Network(nn.Module):
    def __init__(self, p1, p2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(31, 31, bias= True),
            nn.ReLU(),
            nn.Linear(31, 30, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p1),
            nn.Linear(30, 25, bias = True),
            nn.ReLU(),
            nn.Dropout(p= p2),
            nn.Linear(25, 25, bias= True),
            nn.ReLU(),
            nn.Linear(25, 20, bias= True),
            nn.ReLU(),
            nn.Linear(20, 15, bias= True),
            nn.ReLU(),
            nn.Linear(15, 10, bias= True),
            nn.ReLU()
        )
        self.classify = nn.Sequential(
            nn.Linear(10, 1, bias= True),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.classify(self.features(x))

In [3]:
class Data(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x.values, dtype= torch.float32)
        self.y = torch.tensor(y.values, dtype= torch.float32)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, index):
        x, y = self.x[index], self.y[index]
        return x, y

In [4]:
xtrain, ytrain, xtest, ytest = pd.read_csv('./X_train.csv', header= None, index_col = None), pd.read_csv('./y_train.csv', header = None, index_col = None), pd.read_csv('./X_test.csv', header = None, index_col = None), pd.read_csv('./y_test.csv', header = None, index_col = None)
train_data = Data(x= xtrain, y= ytrain)
test_data = Data(x= xtest, y= ytest)
train_loader = DataLoader(train_data, batch_size=10000, shuffle= True)
test_loader = DataLoader(test_data, batch_size= 10000)

In [5]:
model = Network(0.3, 0.6)
model.to('cuda')

Network(
  (features): Sequential(
    (0): Linear(in_features=31, out_features=31, bias=True)
    (1): ReLU()
    (2): Linear(in_features=31, out_features=30, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.3, inplace=False)
    (5): Linear(in_features=30, out_features=25, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.6, inplace=False)
    (8): Linear(in_features=25, out_features=25, bias=True)
    (9): ReLU()
    (10): Linear(in_features=25, out_features=20, bias=True)
    (11): ReLU()
    (12): Linear(in_features=20, out_features=15, bias=True)
    (13): ReLU()
    (14): Linear(in_features=15, out_features=10, bias=True)
    (15): ReLU()
  )
  (classify): Sequential(
    (0): Linear(in_features=10, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [6]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay= 0.001)

In [7]:
for epoch in range(100):
    model.train()
    running_loss = 0.0
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'loss - {running_loss}')

loss - 35.35257989168167
loss - 33.87495255470276
loss - 33.8677094578743
loss - 33.85848927497864
loss - 33.72991555929184
loss - 33.64813190698624
loss - 33.65489053726196
loss - 33.627980172634125
loss - 33.62470942735672
loss - 33.618605971336365
loss - 33.60503560304642
loss - 33.620621621608734
loss - 33.602383613586426
loss - 33.62160325050354
loss - 33.619930267333984
loss - 33.61189216375351
loss - 33.63171523809433
loss - 33.609570264816284
loss - 33.60699778795242
loss - 33.61161643266678
loss - 33.60095018148422
loss - 33.62514239549637
loss - 33.62805622816086
loss - 33.60885101556778
loss - 33.62029939889908
loss - 33.61142945289612
loss - 33.61834704875946
loss - 33.6035760641098
loss - 33.60597348213196
loss - 33.60027098655701
loss - 33.6218239068985
loss - 33.621457517147064
loss - 33.60853308439255
loss - 33.61387610435486
loss - 33.61371463537216
loss - 33.62207514047623
loss - 33.606233060359955
loss - 33.61353290081024
loss - 33.61843299865723
loss - 33.6019566059

In [8]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.to('cuda'), target.to('cuda')
        output = torch.round(model(input))
        all_preds.extend(output.to('cpu'))
        all_labels.extend(target.to('cpu'))

from sklearn.metrics import accuracy_score
print(accuracy_score(all_preds, all_labels))


0.6260255813120276
